In [150]:
from iapws import IAPWS97
import numpy as np
import math

## Static Model Equations


### A. Hydraulic model of single pipe

By law of momentum conservation , hydraulic model can be written as:
$$ C_p = \frac {D^{5} \; \rho_m }{1.25 \times 10^{8} \lambda \; q \; P_1 \; (1+\eta) \; L}  $$

Equation for solving frictional factor $\lambda$:
$$ \lambda = 0.11 \times (\frac{\Delta}{D}+\frac{68}{Re})^{0.25} $$

Reynolds number is written as:
$$ Re = \frac {D \; u \; \rho_m}{\mu} = 354 \frac{q}{D \; \mu}$$

Where:
* $P_1$ - the input pressure of the steam in the pipe, Mpa;
* $\lambda$ - frictional factor
* $q$ - mass flow rate, tph;
* $D$ - the inner diameter of the pipe, mm;
* $L$ - length of the pipe, mm;
* $\rho_m$ - weighted mean density in the pipe, $kg/m^{3}$
* $\eta$ - equivalent length coefficient
* $\Delta$ - equivalent absolute roughness, mm;
* $Re$ - Reynolds number of the steam in the pipe;
* $u$ - characteristic velocity of the steam in the pipe, m/s
* $\mu$ - the mean dynamic viscosity coefficient, Pa*s or kg/(m s)

### B. Thermal model of a single pipe

By law of energy conservation, the static thermal model can be written as follows:

$$ C_T = \frac{278 \; c_p \; q^{2}} {(1+\beta \,) \; q_1 \; L} $$

The amount of heatloss along unit pipe length can be calculated using:

$$ q_l = \frac {T_0 - T_a}
    {\frac{1}{2 \pi \epsilon} \; ln\frac{D_o}{D_i} \; + \frac{1}{\pi \, D_o \, a_w}} $$
    
$a_w$ is determined by:
    $$ a_w = 11.6 + 7 \sqrt {v} $$

In [151]:
"""Hydraulic Model Functions"""
def dviscosity(P):
    """get dynamic viscosity from IAPWS given Pressure (Mpa)"""
    sat_steam=IAPWS97(P=P,x=1)  #saturated steam with known P
    return sat_steam.mu

def sp_heat (T):
    """return specific isobaric heat capacity (Cp); kJ/kg*K"""
    sat_steam=IAPWS97(T=T, x=1)  #saturated steam with known T in Kelvin
    return sat_steam.cp 

def nRe (q, D, mu):
    """Calculate reynolds number"""
    return 353*q/(D * mu)

def ffactor (delta, D, Re):
    """Calculate frictional factor"""
    return 0.11 * ((delta/D)+(68/Re))**0.25

def cp(D, rho, lamda, q, P, eta, L):
    """Calculate from hydraulic model"""
    num = D**5 * rho
    den = 125000000 * lamda * q * P * L * (1-eta)
    return num/den

def aw (v):
    #calculate aw given air velocity; m/s
    return 11.6 + 7*v**0.5
#print ("aw = ", aw(1.11))

def heat_loss (To, Ta, epsilon, Do, Di, aw):
    num = To-Ta
    d1 = np.log(Do/Di)/(2*epsilon*math.pi)
    d2 = 1/(math.pi*(Do)*aw)
    #print (num, d1, d2)
    return (num / (d1 + d2))

def mass_flow (beta, heat_l, L, sp_heat, deltaT):
    return ((1+beta)*heat_l*L)/(278*sp_heat*deltaT)
    
#print (heat_loss(200+273.15, 32+273.15, 0.035, 920, 700, 19))

In [152]:
"""Initialize parameter values"""
eta = 0.2
delta = 0.2
beta = 0.15
epsilon = 0.035
error = 0.3

In [153]:
"""Pipe Specifications"""
#array of pipe lengths
L = np.array([600, 2300, 700, 1300, 1200])
Di = np.array([700, 600, 400, 500, 600])
Do = np.array([720, 630, 426, 529, 630])
insulation = np.array([110, 105, 95, 100, 105])
Do_insul = Do + insulation*2 #Outer diameer of pipe + 2*insulation thickness
#print (Do_insul)

"""Given Measured Data"""
#Define Q preliminary field; flow rate of the ith node
Q = np.array([-117.0, 19.0, 19.0, 79.0, 0.0, 0.0])

#Measured pressure data at nodes 1, 2, 3 and 4; initial guess for node 5 and 6
P = np.array([0.705, 0.679, 0.563, 0.445])
P5 = 0.5*(P[0]+P[1])             #P5 = 0.5*(P1+P2)             initial guess
mx = 0.5*(max(P[2], P[3])+P5)    #P6 = 0.5*( max(P3,P4) + P5)  initial guess
P = np.append(P, [P5,mx]) 

#Measured temp data at nodes 1, 2, 3 and 4; initial guess for node 5 and 6
T = np.array([286+273.15, 195+273.15, 246+273.15, 256+273.15])
mx = max(T[2], T[3])-1.1     #P6 = max(P3,P4) initial guess
T = np.append(T, [T[0]-1.1,mx]) #P5 = P1 initial guess
#print ("temp at nodes:", T)

#Delta T for each pipe segment
dT = np.array([T[0]-T[4], T[4]-T[1], T[5]-T[2], T[5]-T[3], T[4]-T[5]])
print (dT)

#create array of isobaric specific heat at pipeline
cp = np.array([sp_heat(T[0]), sp_heat(T[4]), sp_heat(T[5]), sp_heat(T[5]), sp_heat(T[4])])
#print (cp)


#for heat loss (q_l) calculation, set Ta at inlet temp
#Temperature of pipeline
Ta = np.array([T[0], T[4], T[5], T[5], T[4],])
#print ("temp at pipes:", Ta)

#define atmospheric temp
Tatm = 305.15

#print ("Tatm-Ta: ", Tatm-Ta)
#print ("Do/Di: ", (Do/Di))
#d = Do/Di
#np.log(Do/Di)
#np.log(Do/Di)/(2*math.pi*epsilon) + (1/(math.pi * epsilon ))
#print ("2nd term: ", 1/(math.pi*Do*aw(1.1111)))

#def hl(Do, Di):
#    print ("New func:", np.log(Do/Di))
#    return np.log(Do/Di)
#hl(Do, Di)

#def heat_loss (To, Ta, epsilon, Do, Di, aw):
#    num = To-Ta
#    d1 = np.log(Do/Di)/(2*epsilon*math.pi)
#    d2 = 1/(math.pi*Do*aw)
#    print (num, "\n", d1, "\n", d2)
#    return (num / (d1 + d2))

aw1 = aw(1.11111)
#q_l = heat_loss(Tatm, Ta, epsilon, Do_insul, Di, aw1)
#print (q_l)
#print (278*cp*dT, cp, dT)

[  1.1  89.9   8.9  -1.1  30. ]


In [154]:
"""Step 1: Calculate heat_loss (W/m)"""
#calculate heat loss vector along pipe (W/m)
q_l = heat_loss(Tatm, Ta, epsilon, Do_insul, Di, aw1)
#print (q_l)
#calculate mass flow vector along pipe
q_f = mass_flow(beta, q_l, L, cp, dT)
#print (q_f)

#test q vector; flow rate of each pipe
q = (1+beta)*q_l* L / (278 * cp * dT)
print (dT)
print (q)

Aq = np.dot(A,q)
print (Aq)
print(Q)
obj = Aq + Q
print (obj)
np.sum(obj)

#def mass_flow (beta, heat_l, L, sp_heat, deltaT):
#    return ((1+beta)*heat_l*L)/(278*sp_heat*deltaT)

[  1.1  89.9   8.9  -1.1  30. ]
[ -79.71935425   -3.29662144   -8.89692244  153.08429345   -5.15419595]
[[ -79.71935425    3.29662144    8.89692244 -153.08429345   71.26853686
   149.34156696]]
[-117.   19.   19.   79.    0.    0.]
[[-196.71935425   22.29662144   27.89692244  -74.08429345   71.26853686
   149.34156696]]


2.8421709430404007e-14

## Flow chart of flow rate calculation

<iiimg src="images\flow_chart.png" ,width=700,height=1400>

## Determine matrix "A" and Q



In [155]:
"""Define matrix A based on given steam pipeline network"""
A = np.matrix('1, 0, 0, 0, 0; \
               0, -1, 0, 0, 0; \
               0, 0, -1, 0, 0; \
               0, 0, 0, -1, 0; \
               -1, 1, 0, 0, 1; \
               0, 0, 1, 1, -1')
q = np.array([2, 4, 6, 8, 10])

np.dot(A, q)

a = np.array([1, 2, 3])
b = np.array([10, 20, 30])
c = a + b
np.add(a, b)
#print (c)
np.sum(c)

66